# 작사가 인공지능

오늘 하는 프로젝트는 NLP기반인 인공지능입니다.

직접 인공지능이 많은 데이터를 기반으로 어떻게 언어를 표현하고 있는 그 과정을 수행하는 프로젝트입니다.

## 데이터 다운로드

In [29]:
! mkdir -p ~/aiffel/lyricists/models
! ln -s ~/data ~/aiffel/lyricists/data

ln: failed to create symbolic link '/aiffel/aiffel/lyricists/data/data': Read-only file system


## 데이터 읽어오기

In [20]:
import os
import re
import glob
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

txt_file_path = os.getenv('HOME')+'/aiffel/lyricists/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []


for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 [' There must be some kind of way outta here', 'Said the joker to the thief', "There's too much confusion"]


우리는 문장데이터를 분석할 때 원하는 것만 제공이 되면 좋겠지만 실제로는 그렇지 못하기 때문에 정제나 전처리 과정을 해야합니다.

In [21]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue    
    if sentence[-1] == ":": continue   

    if idx > 9: break   
        
    print(sentence)

 There must be some kind of way outta here
Said the joker to the thief
There's too much confusion
I can't get no relief Business men, they drink my wine
Plowman dig my earth
None were level on the mind
Nobody up at his word
Hey, hey No reason to get excited
The thief he kindly spoke
There are many here among us


평가문항2번

## 데이터 정제

이제 단계별로 수행할 것입니다.

1.모두 소문자화.

2.공백 지우기

3.공백 추가(특수문자 양쪽)

4.공백이 많으면 하나로 통일

5.""외에 다 공백으로 바꿈

6.토큰 갯수가 13개 이상이 아니라면
문장의 시작과 끝에 start와 end추가

이상이면 0

7.아무 문장 넣어서 실행해보기

In [22]:

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence1 = sentence.split(' ')
    
    if len(sentence1) >13:
        return 0
    else:
        sentence = '<start> ' + sentence + ' <end>' # 6
        return sentence



위의 코드를 실제로 진행하고 함수 preprocess_setence에서 만든 
문장의 전처리 과정을 지난 후 저장을 수행합니다.

In [23]:

corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    if preprocessed_sentence ==0:
        pass
    else:
        corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> there must be some kind of way outta here <end>',
 '<start> said the joker to the thief <end>',
 '<start> there s too much confusion <end>',
 '<start> i can t get no relief business men , they drink my wine <end>',
 '<start> plowman dig my earth <end>',
 '<start> none were level on the mind <end>',
 '<start> nobody up at his word <end>',
 '<start> hey , hey no reason to get excited <end>',
 '<start> the thief he kindly spoke <end>',
 '<start> there are many here among us <end>']

데이터 전처리 과정은 끝났으나 단어와 숫자를 매칭시켜주는 토큰화 작업을 해야합니다. 왜냐하면 컴퓨터는 언어가 아닌 숫자로 인식을 하는 스타일이기 때문입니다.

In [24]:


def tokenize(corpus):
   
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    
    tokenizer.fit_on_texts(corpus)
   
    tensor = tokenizer.texts_to_sequences(corpus)   
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2  62 271 ...   0   0   0]
 [  2 118   6 ...   0   0   0]
 [  2  62  17 ...   0   0   0]
 ...
 [  2  75  45 ...   3   0   0]
 [  2  49   5 ...   0   0   0]
 [  2  13 633 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f33cf870b10>


위의 과정이 진짜로 변했나 확인 작업

In [25]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


참고로 LSTM에서는 많은 데이터로 많은 답을 얻기 때문에

train은 첫 문장 ~ 끝에 1개 뺀 문장 구성

trget: 첫번째 단어  뺀 문장으로 구성합니다.

In [26]:

src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:]    
print(src_input[0])
print(tgt_input[0])



[  2  62 271  27  94 546  20  86 742  90   3   0   0   0]
[ 62 271  27  94 546  20  86 742  90   3   0   0   0   0]


데이터 셋 구축

In [18]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

## 평가 데이터 셋 분리

앞서 알려준 사이킷런 패키지를 통해서 8:2로 train과 target를
train과 validation셋으로 분리를 합니다(CS231n 2강파트)

In [27]:

from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True, 
                                                          random_state=34)
                                                         

In [28]:

print('Source Train: ', enc_train.shape)
print('Target Train: ', dec_train.shape)

Source Train:  (124810, 14)
Target Train:  (124810, 14)


## 평가문항 3번

In [16]:
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [17]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = Embedding(vocab_size, embedding_size)
        self.rnn_1 = LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = LSTM(hidden_size, return_sequences=True)
        self.linear = Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

embedding_size = 19
hidden_size = 2048


model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

history = []
epochs = 10

optimizer = tf.keras.optimizers.Adam()

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)


In [52]:

for src_sample, tgt_sample in dataset.take(1): break


model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-1.71207921e-05,  2.04618646e-05, -2.66351435e-05, ...,
          5.77007377e-05,  3.66221866e-05, -6.11733640e-06],
        [ 1.83372867e-05,  4.45840888e-05, -1.13975666e-05, ...,
          8.94761906e-05,  4.75149754e-05, -5.91201533e-05],
        [ 1.53110086e-05,  7.79168258e-05, -4.83180047e-05, ...,
          6.97352589e-05,  4.63235701e-05, -5.45579496e-05],
        ...,
        [-1.80235904e-04,  6.62449602e-05,  2.33214392e-04, ...,
         -9.53052004e-05,  4.37235140e-04,  8.65938346e-05],
        [-2.65075039e-04,  5.50332406e-05,  3.14246456e-04, ...,
         -4.37559320e-05,  5.06745651e-04,  1.45873753e-04],
        [-3.40410508e-04,  4.20210854e-05,  3.88511515e-04, ...,
          5.05564367e-06,  5.78024832e-04,  2.02504729e-04]],

       [[-1.71207921e-05,  2.04618646e-05, -2.66351435e-05, ...,
          5.77007377e-05,  3.66221866e-05, -6.11733640e-06],
        [ 2.99546446e-05,  1.26627617e-06, -5

In [53]:
model.summary()

Model: "text_generator_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      multiple                  228019    
_________________________________________________________________
lstm_6 (LSTM)                multiple                  16941056  
_________________________________________________________________
lstm_7 (LSTM)                multiple                  33562624  
_________________________________________________________________
dense_3 (Dense)              multiple                  24590049  
Total params: 75,321,748
Trainable params: 75,321,748
Non-trainable params: 0
_________________________________________________________________


In [19]:

history = model.fit(enc_train, 
          dec_train, 
          epochs=epochs,
          batch_size=256,
          validation_data=(enc_val, dec_val),
          verbose=1)

Epoch 1/10
550/550 [==============================] - 563s 981ms/step - loss: 3.4055 - val_loss: 2.6118
Epoch 2/10
550/550 [==============================] - 540s 981ms/step - loss: 2.5399 - val_loss: 2.3950
Epoch 3/10
550/550 [==============================] - 540s 982ms/step - loss: 2.3420 - val_loss: 2.2842
Epoch 4/10
550/550 [==============================] - 540s 981ms/step - loss: 2.1995 - val_loss: 2.1852
Epoch 5/10
550/550 [==============================] - 539s 981ms/step - loss: 2.0609 - val_loss: 2.1111
Epoch 6/10
550/550 [==============================] - 540s 983ms/step - loss: 1.9326 - val_loss: 2.0475
Epoch 7/10
550/550 [==============================] - 540s 981ms/step - loss: 1.7906 - val_loss: 1.9964
Epoch 8/10
550/550 [==============================] - 540s 981ms/step - loss: 1.6534 - val_loss: 1.9508
Epoch 9/10
550/550 [==============================] - 540s 981ms/step - loss: 1.5199 - val_loss: 1.9112
Epoch 10/10
550/550 [==============================] - 540s 981m

평가문항 1번

 ###  문장 만드는 과정.
 
1. 입력받은 문장의 텐서를 입력합니다
2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
3. 2에서 예측된 word index를 문장 뒤에 붙입니다
4.모델이 <end>를 예측했거나, max_len에 도달했다면 
      

In [46]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

   
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
     
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [47]:
test_sen = generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)
print(test_sen)

<start> i love the way you lie <end> 


프로젝트 회고

* 처음에 토큰의 갯수를 어떻게 제거해야하는 아이디어를 몰라서 너무나 많이 애를 먹은 것 같습니다. 아직 조건문에 대한 이해도가 너무 낮은 것 같고 NLP는 결국 조건 설정이 가장 중요하다는 것을 알게되었습니다.



* 문장의 길이가 15개 이하를 말했을 때 처음에는 start와 end를 포함하지 않는 것을 생각했으나 막상 프로그램을 돌리고 생각을 해보니 포함하여 15개인 것을 알게 되어 tokenize를 했을 때 13개 이하만 되도록 corpus를 만들었습니다.



* batch_size를 처음에는 256에서 나중에는 2048까지 올렸습니다 왜냐하면 1epoch가 걸리는 시간이 배수까지는 아니더라도 그의 준하게 줄어드는지 아닌지가 궁금해서 입니다. 허나 막상 해보았을 때는 1.1배 정도 밖에 차이가 나지 않아서 너무 단순한 저의 뇌구조에 어이가 없었습니다.

* val_loss를 맞추기 위해서 파라미터들을 다양하게 바꾸면서 최적의 loss을 찾는 법을 배우게 된 것 같습니다.
  이를 통해서 하이퍼파라미터가 주는 변화를 눈으로 볼 수 있었고 그로 인해 생기는 소요시간의 변화, loss의 변화들이 너무나 유의미해서 좋았습니다

* 이번 프로젝트의 핵심은 결국 데이터의 전처리와 셋 구성 과정에서 padding처리, 특수문자제거, 토큰화 등등을 제대로 수행하여 우리가 원하는 인공지능을 구현할 수 있는지에 대해서 입니다.  CV와 NLP기반 인공지능은 뭔가 진짜 사람의 생각을 구현하는 듯해서 신기하고 재미있었습니다.
